In [34]:
import os
import json
import glob
import pandas as pd

In [46]:
SWEEP_PATH = "../autoencoders/othello_5-21/othello-gated_anneal/"

SAE_TRAINER_TYPE = "gated_anneal"

HPARAMS_DICT = {
    "p_anneal" : ["lr", "sparsity_penalty", "anneal_start", "dict_size"],
    "p_anneal_new" : ["lr", "sparsity_penalty", "anneal_start", "dict_size"],
    "standard_new" : ["lr", "l1_penalty", "dict_size"],
    "standard" : ["lr", "l1_penalty"],
    "gated" : ["lr", "l1_penalty", "dict_size"],
    "gated_anneal" : ["lr", "sparsity_penalty", "anneal_start", "dict_size"],
}
HPARAMS = HPARAMS_DICT[SAE_TRAINER_TYPE]

# read in the `results.csv` file from SWEEP_PATH
df = pd.read_csv(SWEEP_PATH + "results.csv")

# filter for SAE type
df = df[df["autoencoder_path"].str.contains(SAE_TRAINER_TYPE + "/")]

# add trainer_id to data frame
df["trainer_id"] = df["autoencoder_path"].map(lambda p: p.split("/")[-2])
df[["trainer_id", "l0", "frac_recovered"]].sort_values("l0")



,trainer_id,l0,frac_recovered
44,trainer44,88.316002,0.991423
48,trainer48,89.310005,0.991205
53,trainer53,90.393005,0.993564
58,trainer58,90.677002,0.996218
49,trainer49,92.388008,0.993828
57,trainer57,92.971001,0.992492
59,trainer59,93.690002,0.995680
54,trainer54,94.259003,0.994364
46,trainer46,98.006004,0.993556
56,trainer56,99.388008,0.989332


In [47]:
#trainer_dirs = df["autoencoder_path"].to_list()
trainer_dirs = [SWEEP_PATH + t_id + "/" for t_id in df["trainer_id"]]

trainer_configs = {}
for t_dir in trainer_dirs:
    trainer_id = t_dir.split("/")[-2]
    with open(t_dir + "/config.json", "r") as f:
        config = json.load(f)
        trainer_configs[trainer_id] = {}
        for h in HPARAMS:
            trainer_configs[trainer_id][h] = config["trainer"][h]

for h in HPARAMS:
    df[h] = df["trainer_id"].map(lambda t_id : trainer_configs[t_id][h])
    
df[["trainer_id", "l0", "frac_recovered"] + HPARAMS].sort_values("l0")

,trainer_id,l0,frac_recovered,lr,sparsity_penalty,anneal_start,dict_size
44,trainer44,88.316002,0.991423,0.003162,0.100000,1000,4096
48,trainer48,89.310005,0.991205,0.003162,0.084140,10000,4096
53,trainer53,90.393005,0.993564,0.003162,0.084140,1000,8192
58,trainer58,90.677002,0.996218,0.003162,0.084140,10000,8192
49,trainer49,92.388008,0.993828,0.003162,0.100000,10000,4096
57,trainer57,92.971001,0.992492,0.003162,0.070795,10000,8192
59,trainer59,93.690002,0.995680,0.003162,0.100000,10000,8192
54,trainer54,94.259003,0.994364,0.003162,0.100000,1000,8192
46,trainer46,98.006004,0.993556,0.003162,0.059566,10000,4096
56,trainer56,99.388008,0.989332,0.003162,0.059566,10000,8192


In [37]:
config

{'trainer': {'trainer_class': 'StandardTrainer',
  'lr': 0.0003162277571391314,
  'l1_penalty': 0.019952621310949326,
  'warmup_steps': 1000,
  'resample_steps': None,
  'device': 'cuda:0',
  'wandb_name': 'StandardTrainer-othello-0'},
 'buffer': {'d_submodule': 512,
  'io': 'out',
  'n_ctxs': 8000.0,
  'ctx_len': 59,
  'refresh_batch_size': 256,
  'out_batch_size': 8192,
  'device': 'cuda:0'}}